In [1]:
from pprint import pformat
import pickle
from pathlib import Path
import os
from IPython.display import display

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from tabulate import tabulate
from tqdm import tqdm

/home/adeel/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
ROOT = Path('/home/adeel/2021 - RAI Toolkit-20210528T125906Z-001/2021 - RAI Toolkit/Country Data/Paraguay')

os.chdir(ROOT)

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('PY2018-SECTIONS.csv')
df.columns = [c.strip().lower() for c in df.columns.to_list()]
df

,sect_id,sect_name,link_id,link_name,speed_flow,traf_flow,acc_class,road_class,clim_zone,surf_class,length,cway_width,shld_width,mt_aadt,nm_aadt,aadt_year,direction,rf,num_rfs,superelev,curvature,sigm_adral,speed_lim,enforcemnt,xnmt,xmt,xfri,hsnew,hsold,hbase,res_modulu,rel_compct,snp_derive,sn,cbr,snp_dry,d0,benkel_def,surf_stren,base_stren,subb_stren,hsubbase,surf_thick,slab_lenth,base_thick,base_modul,cnstr_year,subg_matrl,compmethod,cond_year,roughness,cracks_aca,cracks_acw,cracks_act,ravel_area,phole_num,edgebreak,rut_depth,rutdepth_sd,text_depth,skidresist,drain_cond,faulting,spall_jnts,crackslabs,detercrack,failureskm,grav_thick,last_const,last_surf,last_prvnt,last_rehab,prev_aca,prev_acw,prev_nct,lastgravel,drain_type,altitude,shouldtype,widn_width,edgedrains,nmt_separ,nmtlanes,elanes,calib_item,repcost,condbased,inirough,terrough,rdfosbgr_prreco,rdpvla_prreco,ftcycle_prreco,brdgstr_prreco,trfsgn_prreco,rdfosbgr_resval,rdpvla_resval,ftcycle_resval,brdgstr_resval,trfsgn_resval,rdfosbgr_uselfe,rdpvla_uselfe,ftcycle_uselfe,brdgstr_uselfe,trfsgn_uselfe,rdfosbgr_age,rdpvla_age,ftcycle_age,brdgstr_age,trfsgn_age,compageyear,usflfeunit,id
0,01030401,ATT1E1 1° DE MARZO - CARAGUATAY,1.000304e+08,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,16.342791,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.50,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,7.0,5.0,1.5,1.5,10.0,0.0,10.0,5.0,1.5,0.7,0.45,2,0,0,0,0,0,0,2010,2010,2010,2010,0,0,0,0,7,173,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,1
1,01060102,ATT1E1 ABAI - CAPIITENDY - TARUMA - TUNA,4.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Terciaria,Or,0,1.193982,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.60,2,0,0,0,0,0,0,2018,2018,2018,2018,0,0,0,0,7,232,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,2
2,01071601,ATT1E1 ACCESO A ALTO VERA,7.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Terciaria,Or,0,10.467350,6.1,1.5,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2015,2015,2015,2015,0,0,0,0,7,233,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,3
3,01120103,ATT1E1 ACCESO A ISLA UMBU,9.000000e+00,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,1.219874,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2016,2016,2016,2016,0,0,0,0,7,64,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,4
4,01070401,ATT1E1 ACCESO A PUERTO TRIUNFO,1.200000e+01,NaN,carretera de 2 carriles angostos,Inter-urbano,PY2018,Secundaria,Or,0,14.659920,6.5,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,0,200,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,0,0,2018,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.7,0.45,2,0,0,0,0,0,0,2016,2016,2016,2016,0,0,0,0,7,233,0,0,0,0,0,2,TSBG,0,1,2,12,0,100,0,0,0,0,0,0,0,0,10000,0,0,0,0,0,0,0,0,0,2012,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,00000901,NT4E3 AVDA. ARTIGAS,NaN,NO ES COMPETENCIA DEL MOPC,carretera de 4 carriles,Urbano,PY2018,Primaria,Or,0,6.843380,14.0,0.0,0,0,2017,2,10,1,2,15,0.1,80,1.1,1,1,1,50.0,50,250,15,97,0,2.62,8.0,1,0.0,0,0.35,0.15,0.15,75,0,0,0,0,0,

In [6]:
df.groupby('direction').size().reset_index()

,direction,0
0,0,10
1,1,10
2,2,561


In [14]:
# df.to_csv('processed.csv')

In [8]:
lens = [len(x) for x in df.sect_name.str.split(' - ')]

In [10]:
for i in range(1, max(lens)):
    print(i, sum([l == i for l in lens]))

1 110
2 441
3 29


## Data cleaning

In [8]:
import unicodedata as ud

# https://stackoverflow.com/a/15547803/5908685
def rmdiacritics(char):
    '''
    Return the base character of char, by "removing" any
    diacritics like accents or curls and strokes and the like.
    '''
    desc = ud.name(char)
    cutoff = desc.find(' WITH ')
    if cutoff != -1:
        desc = desc[:cutoff]
        try:
            char = ud.lookup(desc)
        except KeyError:
            pass  # removing "WITH ..." produced an invalid name
    return char

def remove_diacritics(s: str):
    return ''.join(map(rmdiacritics, s))

# 'Palá --> ' + remove_diacritics('Palá')

In [9]:
df.loc[:, 'sect_name'] = df.sect_name.map(str.lower).map(remove_diacritics)
parsed = df.sect_name.str.extract(r'^([^\s]+) (.*)$')
df.loc[:, 'code'] = parsed[0]
df.loc[:, 'name'] = parsed[1]

with pd.option_context('display.max_rows', 100):
    _df = df.name.str.extract(r'(.+)[^\d]$|(.+) ([\d+_])$')
    _df.loc[_df[0].isna(), 0] = _df[_df[0].isna()][1]

df.name = _df[0]

In [10]:
weighted_mean = lambda x: np.average(x, weights=df.loc[x.index, 'length'])
grouped = df[['name', 'length', 'roughness']].groupby('name')
df = grouped.agg({'length': 'sum', 'roughness': weighted_mean})
df['merged_parts'] = grouped.size()

df = df.reset_index()

In [11]:
with pd.option_context('display.max_rows', 500):
    display(df)

,name,length,roughness,merged_parts
0,(ruta acahay - la colmena) - potrero arc,7.871897,3.000000,1
1,14 de febrer,0.076043,3.000000,1
2,1° de marzo - caraguata,16.342791,7.000000,1
3,4 mojones - nemby (acceso sur,7.405948,3.500000,1
4,4 mojones - ytoror,0.712112,4.000000,1
5,abai - capiitendy - taruma - tun,1.193982,3.000000,1
6,acahay a la colmena,29.432978,2.600000,2
7,acahay a ybycu,16.455900,2.600000,1
8,acceso a sta rosa del mbutu,1.843559,3.840000,1
9,acceso a alto ver,10.467350,5.000000,1


In [12]:
len(df)

410

## Simple cases

\<place\> - \<palce\>

In [14]:
# rgx = r'^[\s\w]+? - [\s\w]+?$'
rgx = r'^(?:(?! - ).)* - (?:(?! - ).)*$'
simple_mask = df.name.str.contains(rgx)
df_simple = pd.DataFrame(df[simple_mask])
df_simple

,name,length,roughness,merged_parts
2,1° de marzo - caraguata,16.342791,7.00,1
3,4 mojones - nemby (acceso sur,7.405948,3.50,1
4,4 mojones - ytoror,0.712112,4.00,1
26,acceso sur - villeta,9.290712,4.65,2
30,aeropuerto - m. r. alons,5.744811,7.00,1
...,...,...,...,...
402,ypane - desv guarambar,0.663527,5.00,1
403,ytororo - cervepa,1.683191,4.00,1
404,ytororo - desv. guarambare (acceso sur,3.789801,3.80,1
405,ytororo - ypan,3.657007,5.00,1


In [17]:
# with open('test.pkl', 'wb') as f:
#     pickle.dump(df_simple, f)

with open('test.pkl', 'rb') as f:
    df_simple = pickle.load(f)

In [51]:
np.mean(simple_mask)

0.4121951219512195

### Parse endpoints

In [54]:
endpoint_1 = [p for (_, (p, _)) in df_simple.name.str.split(' - ').items()]
endpoint_2 = [p for (_, (_, p)) in df_simple.name.str.split(' - ').items()]
df_simple.loc[:, 'endpoint_1'] = list(map(str.strip, endpoint_1))
df_simple.loc[:, 'endpoint_2'] = list(map(str.strip, endpoint_2))

In [55]:
df_simple[df_simple.endpoint_1.str.contains(r'\d$', regex=True)]

,name,length,roughness,merged_parts,endpoint_1,endpoint_2
143,cruce ruta 5 - bella vista norte,97.371277,1.858940,2,cruce ruta 5,bella vista norte
221,km7 - supercarreter,6.856121,2.000000,1,km7,supercarreter
293,py01 km 15 - reduto cuartel de la victori,7.163970,5.000000,1,py01 km 15,reduto cuartel de la victori
296,ramal ruta 8 - santa catalin,3.800467,3.000000,1,ramal ruta 8,santa catalin
306,ruta 2 - aldana canad,12.080570,7.000000,1,ruta 2,aldana canad
307,ruta 3 - ex caballeri,1.853524,4.000000,1,ruta 3,ex caballeri
309,ruta 7 km 14 - cedrales,20.632039,3.000000,2,ruta 7 km 14,cedrales
310,ruta 7 km 4 - pte franco,8.310180,3.215348,2,ruta 7 km 4,pte franco
311,ruta 9 - loma plat,20.985889,4.200000,1,ruta 9,loma plat
312,ruta 9 - neulan,20.671471,4.220000,1,ruta 9,neulan


In [24]:
df_simple.to_csv('simple_cases.csv', index=False)